In [1]:
# Activate the environment using the path to the Macro repository
macro_repo_path = dirname(@__DIR__)
using Pkg; Pkg.activate(macro_repo_path)

  Activating project at `~/Code/Macro`


In [2]:
using Macro
using HiGHS
using CSV
using DataFrames
using JSON3

In this tutorial, we start from a single zone electricity system with four resource clusters: utility scale solar PV, land-based wind power generation, natural gas combined cycle power plants, and electricity storage. 

We consider three commodities: electricity, natural gas, and $\text{CO}_2$. Initially, hydrogen is modeled exogenously, including a fixed hydrogen demand as part of the electricity demand.

We model a greenfield scenario with a carbon price of 250 $/ton.

***Note: We use the default units in MACRO: MWh for energy vectors, metric tons for other commodities (e.g., $\text{CO}_2$) and dollars for costs***

We first load the inputs:

In [3]:
system = Macro.load_system("one_zone_electricity_only");

┌ Info: Loading JSON data from /Users/fpecci/Code/Macro/tutorials/one_zone_electricity_only/system_data.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from one_zone_electricity_only/system_data.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from one_zone_electricity_only/system/nodes.json
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading columns [:natgas_SE] from CSV data from one_zone_electricity_only/system/fuel_prices.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:8
┌ Info: Loading CSV data from one_zone_electricity_only/system/fuel_prices.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading columns [:Demand_MW_z1] from CSV data from one_zone_electricity_only/system/demand.csv
└ @ Macro /Users/fpecci/Code/Macro/src/load_inputs/file_io/csv.jl:8
┌ Info: Loading CSV data 

system.settings.Scaling = false


Then, we generate the Macro capacity expansion model:

In [4]:
model = Macro.generate_model(system)

┌ Info: Starting model generation
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:15
┌ Info: Defining available capacity
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:18
┌ Info: Generating planning model
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:21
┌ Info: Generating operational model
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:24
┌ Info: Model generation complete, it took 5.108589172363281 seconds
└ @ Macro /Users/fpecci/Code/Macro/src/generate_model.jl:29


A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: JuMP.AffExpr
├ num_variables: 105133
├ num_constraints: 297861
│ ├ JuMP.AffExpr in MOI.EqualTo{Float64}: 43801
│ ├ JuMP.AffExpr in MOI.GreaterThan{Float64}: 26281
│ ├ JuMP.AffExpr in MOI.LessThan{Float64}: 122647
│ ├ JuMP.VariableRef in MOI.EqualTo{Float64}: 5
│ └ JuMP.VariableRef in MOI.GreaterThan{Float64}: 105127
└ Names registered in the model
  └ :eFixedCost, :eVariableCost, :vREF

Next, we set the optimizer. Note that we are using the open-source LP solver [HiGHS](https://highs.dev/), alternatives include the commerical solvers [Gurobi](https://www.gurobi.com/), [CPLEX](https://www.ibm.com/products/ilog-cplex-optimization-studio), [COPT](https://www.copt.de/).

In [5]:
Macro.set_optimizer(model, HiGHS.Optimizer);

Finally, we solve the capacity expansion model:

In [6]:
Macro.optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-08, 3e+05]
  Cost   [1e-01, 6e+07]
  Bound  [1e+00, 1e+00]
  RHS    [1e+05, 3e+05]
Presolving model
162466 rows, 92389 cols, 513676 nonzeros  0s
148838 rows, 78761 cols, 491004 nonzeros  0s
Presolve : Reductions: rows 148838(-43891); columns 78761(-26372); elements 491004(-158060)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 8760(2.24675e+08) 0s
      32546     6.4134099335e+10 Pr: 12367(1.56206e+08) 5s
      63518     8.5696070667e+10 Pr: 4073(3.65187e+07) 10s
      81205     9.4457002031e+10 Pr: 5649(2.23468e+08); Du: 0(9.74435e-08) 15s
      95349     9.6967642732e+10 Pr: 3200(2.95895e+07) 21s
     100731     9.7122334175e+10 Pr: 0(0); Du: 0(2.15404e-08) 22s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Sim

And extract the results:

In [7]:
capacity_results = Macro.get_optimal_asset_capacity(system)

Row,asset,type,capacity,additions,retirements
,Symbol,Symbol,Float64,Float64,Float64
1,battery_SE,Battery,107497.0,107497.0,0.0
2,SE_naturalgas_ccavgcf_moderate_0,ThermalPower{NaturalGas},1.23484e5,215.505,0.0
3,SE_utilitypv_class1_moderate_70_0_2_1,VRE,335937.0,335937.0,0.0
4,SE_landbasedwind_class4_moderate_70_1,VRE,2.70411e5,2.70411e5,0.0


The total system cost (in dollars) is:

In [8]:
Macro.objective_value(model)

9.712233417508975e10

and the total emissions (in metric tonnes) are:

In [9]:
co2_node_idx = findfirst(isa.(system.locations,Node{CO2}).==1)
Macro.value(sum(system.locations[co2_node_idx].operation_expr[:emissions]))

8.156522666248773e7

**Task :** Set a strict net-zero $\text{CO}_2$ cap by removing the slack allowing constraint violation for a penalty. This can be done by deleting the field `price_unmet_policy` from the $\text{CO}_2$ node in file `one_zone_electricity_only/system/nodes.json`

Re-load the model with the new inputs and solve it to obtain the capacity results. Is there any difference?

In [11]:
Macro.objective_value(model)

9.712233417508975e10

In [12]:
co2_node_idx = findfirst(isa.(system.locations,Node{CO2}).==1)
Macro.value(sum(system.locations[co2_node_idx].operation_expr[:emissions]))

8.156522666248773e7

In [13]:
-Macro.shadow_price(system.locations[co2_node_idx].constraints[1].constraint_ref[1])

200.0